# Dog Classifying Model

## Refer :
https://www.kaggle.com/code/aashishsekar/dog-classifying-model/notebook for Cell output‎‎‎‎‎‎‎‎‎‎‎‎‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ 
          ‎‎‎‎‎‎ ‎‎‎     https://www.kaggle.com/code/aashishsekar/dog-classifying-model/output for the Trained Model (as it is a huge file to be uploaded in github)

In [ ]:
#importing the libararies required
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from keras.models import Sequential
import numpy as np 
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
#path of the dataset
train_dir='/kaggle/input/stanford-dogs-dataset/images/Images'

In [ ]:
# adding preprocessing layer to  VGG16
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# not train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
folders = glob('/kaggle/input/stanford-dogs-dataset/images/Images/*')
len(folders)

In [ ]:
#flattening the vgg-16 model
x = Flatten()(vgg.output)
#activating dense layers tot he model
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# structure of the model
model.summary()

In [ ]:
#compiling the model
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
#to rescale the training images
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#to rescale the validation images
valid_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)

#to create the training data
training_model = train_datagen.flow_from_directory(  
    train_dir,  
    target_size= (224, 224), 
    color_mode= 'rgb',
    batch_size= 32,  
    class_mode= 'categorical',
    subset='training',
    shuffle= True, 
    seed= 1337
) 
#to create the validating data
validating_model = valid_datagen.flow_from_directory(
    train_dir,
    target_size= (224, 224),
    color_mode= 'rgb',
    batch_size= 32,  
    class_mode= 'categorical',
    subset='validation',
    shuffle= True, 
    seed= 1337
)

In [ ]:
print(len(training_model))
print(len(validating_model))

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Checkpoint and monitoring val accuracy
mc = ModelCheckpoint(
    'dog_class_model.h5',
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True,
)


In [ ]:
#training the model
r = model.fit(
    training_model, 
    epochs=100,
    steps_per_epoch=len(training_model),
    validation_data=validating_model,  
    validation_steps=len(validating_model),  
    verbose = 2, 
    callbacks=[mc],
    shuffle = True    
)
!mkdir -p saved_model
model.save('saved_model/dog_class_model.h5')

In [ ]:
# plotting loss occured
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
# plotting the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model
#loading the model and to check the structure
model1 = load_model('./dog_class_model.h5',compile=False) 
model1.summary()